In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
import json
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

c:\Users\mo\Documents\Organisations\RUG\Language-Technology-Project


### Load the data


In [2]:

def read_data(path: str) -> List[dict]:
    # parse the content of the json file
    text = open(path, 'r').read()
    # parse jsonl
    data = [json.loads(jline) for jline in text.split('\n') if jline]
    return data

def write_data(data: List[dict], path: str) -> None:
    with open(path, 'w') as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write('\n')


In [4]:
path = "./datasets/threads_light.jsonl"
data = read_data(path)

1024


In [11]:
data[0]

{'distinguished': None,
 'retrieved_on': 1440530254,
 'stickied': False,
 'gilded': 0,
 'author_flair_css_class': None,
 'comments': [{'distinguished': None,
   'retrieved_on': 1441421589,
   'level': 0,
   'gilded': 0,
   'parent_id': 't3_3i1y4z',
   'body': 'When I was little, my dad would deliberately make me angry when playing football. The lesson he was trying to teach me was to direct my anger in the right way, to stop me from wanting to give up, and to do my best. That childlike sense of anguish I got from being angry and things being unfair gave me a lot of motivation.\n\nIf you had said "There is no good reason to *react angrily*" then you would have had a stronger case. As it stands, when it comes to social injustice, political issues, or just the unfair things that happen to people in life *feeling angry* is the starting point for something to change. A million children dying of starvation every day and because we\'re so desensitised we don\'t do anything about it. It\'s a t

In [6]:
print(f"Key values: {data[0].keys()}")
print(f"Comment keys: {data[0]['comments'][0].keys()}")

Key values: dict_keys(['distinguished', 'retrieved_on', 'stickied', 'gilded', 'author_flair_css_class', 'comments', 'secure_media_embed', 'ups', 'media_embed', 'subreddit', 'subreddit_id', 'from', 'selftext', 'downs', 'secure_media', 'link_flair_css_class', 'num_comments', 'score', 'created_utc', 'title', 'delta', 'domain', 'from_kind', 'archived', 'created', 'link_flair_text', 'is_self', 'over_18', 'thumbnail', 'from_id', 'author_flair_text', 'edited', 'id', 'permalink', 'saved', 'name', 'urls', 'quarantine', 'media', 'author', 'url', 'hide_score'])
Comment keys: dict_keys(['distinguished', 'retrieved_on', 'level', 'gilded', 'parent_id', 'body', 'ups', 'author', 'controversiality', 'link_id', 'subreddit', 'score', 'subreddit_id', 'author_flair_text', 'edited', 'id', 'urls', 'children', 'author_flair_css_class', 'created_utc'])


In [32]:
def format_data(data):

    result = []
    for entry in data:
        c = {
            "head": entry["title"],
            "comments": []
        }
        if len(entry["comments"]) > 0:
            stack = [entry["comments"][0]]
            while stack:
                comment = stack.pop()
                for reply in comment["children"]:
                    stack.append(reply)
                c["comments"].append(comment["body"])
        result.append(c)
    return result

def show_data(data):
    # Should truncate the data to 10 and put ... in between
    string = ""
    for i in range(min(10, len(data))):
        string += "Head: " + data[i]["head"]
        
        for j in range(min(3, len(data[i]["comments"]))):
            string += f"\n Comment {j}: " + data[i]["comments"][j][:50] + "..."
        
        string += "\n ...\n" if len(data[i]["comments"]) > 3 else "\n"
        string += "\n"
    string += "..." if len(data) > 10 else ""
    print(string)

formatted_data = format_data(data)
show_data(formatted_data)

Head: CMV: I don't believe there is any good reason to ever feel angry about anything. I don't think it can ever help you even a little bit.
 Comment 0: When I was little, my dad would deliberately make ...

Head: I Believe Junior Soldiers Are Grossly Overpaid
 Comment 0: Your submission has been automatically removed bec...

Head: I think they should "cultivate" a more laid back culture here cmv
 Comment 0: The problem is less strict subreddits run into pro...
 Comment 1: I'm subbed to r/braveryjerk; I'm quite aware of wh...

Head: CMV: Toronto is the best city in North America
 Comment 0: Hey i live in toronto... it's definitely not the b...
 Comment 1: &gt;  I'd give that to either SF (where I've lived...
 Comment 2: &gt; I actually think Toronto has better weather!
...
 ...

Head: CMV: Students get more out of standard-level courses than they do out of AP courses.
 Comment 0: If your teachers focus on "teaching the test" as m...
 Comment 1: I suppose? My issue is that the curriculu